<a href="https://colab.research.google.com/github/rohit9650/NLP/blob/master/Assignment%202/nlp_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
NLP assignment 2
Problem statement and dataset : https://github.com/rohit9650/NLP/tree/master/Assignment%201

author : Rohit Singh (MCS201812)
"""

In [0]:
import numpy as np 
import nltk
from nltk import bigrams    
from nltk.corpus import stopwords
nltk.download('stopwords')
import os
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from sklearn.decomposition import TruncatedSVD
import random
from scipy import spatial

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# downloading corpus
! wget -O corpus.zip "https://github.com/rohit9650/NLP/blob/master/Assignment%201/corpus.zip?raw=true" 

--2019-03-15 08:56:16--  https://github.com/rohit9650/NLP/blob/master/Assignment%201/corpus.zip?raw=true
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/rohit9650/NLP/raw/master/Assignment%201/corpus.zip [following]
--2019-03-15 08:56:17--  https://github.com/rohit9650/NLP/raw/master/Assignment%201/corpus.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rohit9650/NLP/master/Assignment%201/corpus.zip [following]
--2019-03-15 08:56:17--  https://raw.githubusercontent.com/rohit9650/NLP/master/Assignment%201/corpus.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... con

In [0]:
# listing files in directory "corpus.zip" must be there now
os.listdir("/content/")

['.config', 'corpus.zip', 'sample_data']

In [0]:
# unzipping the corpus
!unzip '/content/corpus.zip'

Archive:  /content/corpus.zip
   creating: corpus/
  inflating: corpus/filename0000.txt  
  inflating: corpus/filename0114.txt  
  inflating: corpus/filename0001.txt  
  inflating: corpus/filename0002.txt  
  inflating: corpus/filename0003.txt  
  inflating: corpus/filename0004.txt  
  inflating: corpus/filename0005.txt  
  inflating: corpus/filename0006.txt  
  inflating: corpus/filename0007.txt  
  inflating: corpus/filename0008.txt  
  inflating: corpus/filename0009.txt  
  inflating: corpus/filename0010.txt  
  inflating: corpus/filename0011.txt  
  inflating: corpus/filename0012.txt  
  inflating: corpus/filename0013.txt  
  inflating: corpus/filename0014.txt  
  inflating: corpus/filename0015.txt  
  inflating: corpus/filename0016.txt  
  inflating: corpus/filename0017.txt  
  inflating: corpus/filename0018.txt  
  inflating: corpus/filename0019.txt  
  inflating: corpus/filename0020.txt  
  inflating: corpus/filename0021.txt  
  inflating: corpus/filename0022.txt  
  inflating: 

In [0]:
def get_files(dir_path):
    """
    return the list of all files present in dir_path
    """
    files = []
    for file in sorted(os.listdir(dir_path)):
        files.append(file)
    return files


In [0]:
def preprocess_data(doc_set):
    """
    Input  : document list
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : preprocessed text as list and list of list
    """
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    # list for tokenized documents in loop
    text_list = []
    text_list_of_list = []
    # loop through document list
    for file in doc_set:
        file_path = '/content/corpus' + "/" + file
        fp = open(file_path,"r")
        i = fp.read()
        # remove numbers
        i = re.sub(r'\d+', '', i)
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        # add tokens to list
        text_list = text_list + stemmed_tokens
        text_list_of_list.append(stemmed_tokens)
    return text_list, text_list_of_list


In [0]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)

    # Key:Value = Word:Index
    vocab_to_index = { word:i for i, word in enumerate(vocab) }

    # Create bigrams from all words in corpus
    bi_grams = list(bigrams(corpus))
    # Frequency distribution of bigrams ((word1, word2), num_occurrences)
    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))
    # Initialise co-occurrence matrix

    co_occurrence_matrix = np.zeros((len(vocab), len(vocab)))

    # Loop through the bigrams in the frequency distribution, noting the 
    # current and previous word, and the number of occurrences of 
    # the bigram.
    # Get the vocab index of the current and previous words.
    # Put the number of occurrences into the appropriate element 
    # of the array.
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]
        pos_current = vocab_to_index[current]
        pos_previous = vocab_to_index[previous]
        co_occurrence_matrix[pos_current][pos_previous] = count
        co_occurrence_matrix[pos_previous][pos_current] = count
    co_occurrence_matrix = np.matrix(co_occurrence_matrix)
    
    return co_occurrence_matrix, vocab_to_index


In [0]:
path = '/content/corpus'
corpus_as_list, corpus_as_list_of_list = preprocess_data(get_files(path))
m, vocab_to_index = co_occurrence_matrix(corpus_as_list)

#%%
word_indices = [random.randint(1,754) for x in range(10)]
words = []
for x in range(len(word_indices)):
    for word, i in vocab_to_index.items():
        if i == word_indices[x]:
            words.append(word)

#for measuring the similarity between these words
word1 = 'away'
word2 = 'east'

print('shape of word co-occurrence matrix before redcution : {}'.
      format(m.shape))
#word vectors before reduction 
for x in range(len(words)):
    print(words[x] + ' : ')
    print(m[word_indices[x]][:])

svd = TruncatedSVD(n_components=15, n_iter=7, random_state=42)
m = svd.fit_transform(m)
print('shape of word co-occurrence matrix after redcution : {}'.
      format(m.shape))


#word vectors After reduction 
for x in range(len(words)):
    print(words[x] + ' : ')
    print(m[word_indices[x]][:])

word1_vec = m[vocab_to_index[word1]][:]
word2_vec = m[vocab_to_index[word2]][:]
similarity = 1 - spatial.distance.cosine(word1_vec, word2_vec)

print('similarity between words using lsi : {}'.format(similarity))


shape of word co-occurrence matrix before redcution : (753, 753)
hail : 
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [0]:
from gensim.models import Word2Vec

documents = corpus_as_list_of_list

model = Word2Vec(
        documents,
        size=15,
        window=10,
        min_count=2,
        workers=10)
model.train(documents, total_examples=len(documents), epochs=10)
vocab = list(model.wv.vocab.keys())
print(len(vocab))
for word in words:
    if word in vocab:
        print(word + ' : ')
        print(model[word])

print('similarity between words using word2vec : {}'.
      format(model.similarity(word1, word2)))

450
hail : 
[ 0.20906258  0.05950342 -0.07988214 -0.06658026  0.13358952  0.00035428
 -0.05969271 -0.15774333  0.01821885 -0.13289729 -0.04284043  0.29175216
  0.05973857 -0.00637272  0.01252111]
bicycl : 
[ 0.5771764   0.20837347 -0.15113893 -0.13065235  0.3756109  -0.02872476
 -0.19925728 -0.4080223   0.05446416 -0.29256335 -0.14098185  0.6550672
  0.18975034 -0.04173634  0.03107618]
diego : 
[ 0.6371439   0.22904915 -0.17735295 -0.11534565  0.40158758 -0.04777213
 -0.23109686 -0.42678824  0.0756458  -0.27436277 -0.11867758  0.72777426
  0.19806175 -0.04336616  0.0734105 ]
penetr : 
[ 0.30855244  0.10343279 -0.07725293 -0.08205175  0.19311269 -0.04608217
 -0.10704585 -0.27726462  0.04799226 -0.19077039 -0.07453614  0.39717054
  0.11385931  0.03085589  0.02081998]
hotel : 
[ 1.1256202   0.37362963 -0.2911832  -0.21681093  0.7195874  -0.02019183
 -0.39723948 -0.836242    0.16197456 -0.5240559  -0.24800105  1.3288405
  0.31662232 -0.06050669  0.08705126]
similarity between words using w

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


#**Question 3**

##*Is factoring a matrix same as back-propagation error mechanism to update weights. If yes or no, why?*

#**Answer  : **
A word embedding method which uses back-propagation introduced by Mikolov et al.,  is implicitly factorizing
a word-context matrix, whose cells are the pointwise mutual information (PMI) of
the respective word and context pairs, shifted by a global constant.

###Skip-Gram with Negative Sampling (SGNS)

####**Setting and Notation** 
The skip-gram model assumes a corpus of words *w* ∈ $V_W$ and their
contexts *c* ∈ $V_C$ , where $V_W$ and $V_C$ are the word and context vocabularies. In 
the words come from an unannotated textual corpus of words *$w_1$, $w_2$, . . . , $w_n$* (typically n is in
the billions) and the contexts for word $w_i$ are the words surrounding it in an L-sized window
$w_L$, . . . , $w_{i-1}$, $w_{i+1}$, . . . , $w_{i+L}$. Other definitions of contexts are possible. We denote the
collection of observed words and context pairs as **D**. We use *#(w, c)* to denote the number of times
the pair* (w, c)* appears in **D**. Similarly, 
* \begin{equation*}
\#(w) = \sum_{c'\in V_c} \#w(w,c')
\end{equation*} 
* \begin{equation*}
\#(c) = \sum_{w'\in V_w} \#w(w',c)
\end{equation*} 
are the number of times *w* and *c* occurred in **D**, respectively.


Each word *w* ∈ $V_W$ is associated with a vector $\vec{w}$ ∈ $R^d$
and similarly each context *c* ∈ $V_C$ is
represented as a vector $\vec{c}$ ∈ $R^d$
, where *d* is the embedding’s dimensionality. The entries in the
vectors are latent, and treated as parameters to be learned. We sometimes refer to the vectors $\vec{w}$ as
rows in a |$V_W$ | × *d* matrix *W*, and to the vectors $\vec{c}$ as rows in a |$V_C$ | × *d* matrix *C*. In such cases, $W_i$
($C_i$) refers to the vector representation of the ith word (context) in the corresponding vocabulary.
When referring to embeddings produced by a specific method x, we will usually use $W^x$
and $C^x$
explicitly, but may use just *W* and *C* when the method is clear from the discussion.

####**SGNS’s Objective** 


\begin{equation*}
l =  \sum_{w\in V_w}  \sum_{c\in V_c}  \#(w, c) ~(log ~σ( ~\vec{w} · \vec{c}) + k · \mathop{\mathbb{E}}_{c_N ∼P_D} [log ~σ(−~\vec{w} · \vec{c_N} )]) 
\end{equation*}

###SGNS as Implicit Matrix Factorization

SGNS embeds both words and their contexts into a low-dimensional space $R^d$
, resulting in the
word and context matrices $W$ and $C$. The rows of matrix $W$ are typically used in NLP tasks (such
as computing word similarities) while $C$ is ignored. It is nonetheless instructive to consider the
product *$W·C^T$* = $M$. Viewed this way, SGNS can be described as factorizing an implicit matrix
$M$ of dimensions $|V_W | × |V_C | $into two smaller matrices.


Which matrix is being factorized? A matrix entry $M_{ij}$ corresponds to the dot product $W_i
· C_j =
\vec{w_i}
· \vec{c_j}$. Thus, SGNS is factorizing a matrix in which each row corresponds to a word $w ∈ V_W $,
each column corresponds to a context $c ∈ V_C$ , and each cell contains a quantity $f(w, c)$ reflecting
the strength of association between that particular word-context pair. Such word-context association
matrices are very common in the NLP and word-similarity literature. That said, the
objective of SGNS  does not explicitly state what this association metric is. What can
we say about the association function $f(w, c)$? In other words, which matrix is SGNS factorizing?

####**Characterizing the Implicit Matrix** 

Consider the global objective above. For sufficiently large dimensionality $d$ (i.e. allowing
for a perfect reconstruction of $M$), each product $\vec{w_i}
· \vec{c_j}$ can assume a value independently of the
others. Under these conditions, we can treat the objective $l$ as a function of independent $\vec{w_i}
· \vec{c_j}$terms,
and find the values of these terms that maximize it.
We begin by rewriting global objective:

\begin{equation*}
l =  \sum_{w\in V_w}  \sum_{c\in V_c}  \#(w, c) ~(log ~σ( ~\vec{w} · \vec{c}) )+ 
\sum_{w\in V_w}  \sum_{c\in V_c}  \#(w, c)~(k · \mathop{\mathbb{E}}_{c_N ∼P_D} [log ~σ(−~\vec{w} · \vec{c_N} )]) 
\end{equation*}

\begin{equation*}
  =  \sum_{w\in V_w}  \sum_{c\in V_c}  \#(w, c) ~(log ~σ( ~\vec{w} · \vec{c}) )+ 
\sum_{w\in V_w}    \#(w)~(k · \mathop{\mathbb{E}}_{c_N ∼P_D} [log ~σ(−~\vec{w} · \vec{c_N} )]) 
\end{equation*}  

and explicitly expressing the expectation term:

\begin{equation*}
\mathop{\mathbb{E}}_{c_N ∼P_D} [log ~σ(−~\vec{w} · \vec{c_N} )]  =   \sum_{c_N\in V_c}  {\frac{\#(c_N)}{|D|}} ~(log ~σ(- ~\vec{w} · \vec{c}) )
\end{equation*}

\begin{equation*}
=  {\frac{\#(c_N)}{|D|}} ~(log ~σ(- ~\vec{w} · \vec{c}) ) +   \sum_{c_N\in V_c - c}  {\frac{\#(c_N)}{|D|}} ~(log ~σ(- ~\vec{w} · \vec{c}) )
\end{equation*}

Combining equations reveals the local objective for a specific $(w, c)$ pair : 

\begin{equation*}
l(w,c) = \#(w,c)~(log ~σ(~\vec{w} · \vec{c}) ) -~k~. \#(w) {\frac{\#(c)}{|D|}} ~(log ~σ(- ~\vec{w} · \vec{c}) )
\end{equation*}

To optimize the objective, we define $x = \vec{w} · \vec{c}$ and find its partial derivative with respect to $x$ :

\begin{equation*}
\frac{\partial l}{\partial x} = \#(w,c)~σ(-x)  -~k~. \#(w) ~{\frac{\#(c)}{|D|}} ~σ(x)
\end{equation*}

We compare the derivative to zero, and after some simplification, arrive at:

\begin{equation*}
e^{2x} - ~\left ( ~ \frac{\#(w,c)}{~k~. \#(w) ~{\frac{\#(c)}{|D|}}} ~ - ~1 \right )e^x - 
~\left ( ~ \frac{\#(w,c)}{~k~. \#(w) ~{\frac{\#(c)}{|D|}}} ~ - ~1 \right ) = 0
\end{equation*}

If we define $y = e^x$
, this equation becomes a quadratic equation of $y$, which has two solutions,
$y = −1$ (which is invalid given the definition of $y$) and :

\begin{equation*}
y = ~\left ( ~ \frac{\#(w,c)}{~k~. \#(w) ~{\frac{\#(c)}{|D|}}} ~ \right ) = 
\frac{\#(w,c) . |D|}{\#w.\#c}.\frac{1}{k}
\end{equation*}

Substituting $y$ with $e^x$
and $x$ with $\vec{w}.\vec{c}$ reveals :

\begin{equation*}
 \vec{w}.\vec{c} = log \left (  \frac{\#(w,c) . |D|}{\#w.\#c}.\frac{1}{k} \right) = 
  log \left (  \frac{\#(w,c) . |D|}{\#w.\#c} \right) - log k
\end{equation*}


Interestingly, the expression $ log \left (  \frac{\#(w,c) . |D|}{\#w.\#c} \right) $
is the well-known pointwise mutual information
($PMI$) of $(w, c)$, which we discuss in depth below.
Finally, we can describe the matrix $M$ that SGNS is factorizing :

\begin{equation*}
M_{ij}^{SGNS} = W_i.C_j = \vec{w_i}.\vec{c_j} = P M I (w_i, c_j) -log k
\end{equation*}



*so, factoring a matrix is same as back-propagation error mechanism to update weights.*


#**Question 4**

##*Discuss (about 100 words) how LSI word vectors could be improved to be close to s.t *

$||wv_{LSI}^2 - wv_{w2v}^2 ||  {\simeq0}$

#**Answer  : **

LSI word vectors are based on the co-occurrence matrix. If we see closely the co-occurrence matrix has entries $M_{ij}$ = \# $word_i$ occurred with $word_j$. This type of relationship would be good if we are taking only the syntactic aspect of word vectors. On the other hand, word2vec also incorporate the semantic nature of words, therefore,  we need a way to incrporate this relationship that into our co-occurrence matrix. This can be done by using *n-gram* models to build our matrix.
The Glove model does this.